| Aplicativo | Play Store ID           | App Store ID |
| ---------- | ----------------------- | ------------ | 
| Claro      | com.nvt.cs              | 526829675    |
| TIM        | br.com.timbrasil.meutim | 668591218    |
| Vivo       | br.com.vivo             | 475915064    |



In [1]:
import requests
import time
import pandas as pd
from google_play_scraper import Sort, reviews

Tem como pegar versão do app? ou outras info além de usuario, nota e comentário?

## Extração de informações gerais do app

In [4]:
from google_play_scraper import app
result = app(
    'br.com.vivo',
    lang='pt', # defaults to 'en'
    country='br' # defaults to 'us'
)
result

{'title': 'Vivo',
 'description': 'Conheça o app Vivo! Além de gerenciar os seus serviços contratados, você pode aproveitar diversos produtos inovadores e ainda ganhar benefícios no app. \r\nPara se cadastrar é muito fácil, basta seguir as seguintes etapas.\r\n\r\nInsira seu CPF e e-mail\r\nApós isso, siga as instruções e crie sua senha numérica de 6 dígitos\r\n\r\nPronto, é só aproveitar!\r\n\r\nConheça mais sobre o app:\r\n\r\nConsultas e pagamentos\r\nNa área Assinaturas, é possível consultar suas faturas e realizar o pagamento quando for necessário, oferecendo a você toda a nossa praticidade.\r\n\r\nCompartilhe e envie internet\r\nSó pelo app, clientes Vivo Pré compartilham ou enviam internet para outros números Vivo. É rápido e simples.\r\n\r\nLoja\r\nAs melhores ofertas selecionadas para você. Aqui você vai poder assinar novos serviços Vivo ou de parceiros, além de comprar produtos diversos com ofertas exclusivas.\r\n\r\nBônus\r\nO lugar perfeito pra garantir bônus na sua interne

## Coleta de Comentários

**Dados Play Store**

In [13]:
qtd_comentarios = 1000

dados_android = []

apps = {
    "Claro": "com.nvt.cs",
    "TIM": "br.com.timbrasil.meutim",
    "Vivo": "br.com.vivo"
}

for app, app_id in apps.items():
    result, _ = reviews(
        app_id,
        lang='pt',
        country='br',
        sort=Sort.NEWEST,
        count=qtd_comentarios
    )
        
    for review in result:
        dados_android.append({
            "app": app,
            "system": "Android",
            "user": review.get("userName", None),
            "score": review.get("score", None),
            "content": review.get("content", None),
            "app_version": review.get("appVersion", None),
            "date": review.get("at", None),
            "reply_content": review.get("replyContent", None),
            "reply_date": review.get("repliedAt", None)
        })

comentarios_android = pd.DataFrame(dados_android, columns=["app", "system", "user", "score", "content",
                                                           "app_version", "date", "reply_content", "reply_date"])

comentarios_android.head()

,app,system,user,score,content,app_version,date,reply_content,reply_date
0,Claro,Android,Neide Alves,4,nota 10❤️,18.6.0,2025-05-26 21:07:03,None,NaT
1,Claro,Android,Nilson Costa rodrigues,5,Nota 10,18.7.0,2025-05-26 20:26:28,None,NaT
2,Claro,Android,Josefina Rodrigues,5,muito eficiente,18.6.0,2025-05-26 20:21:47,None,NaT
3,Claro,Android,Miriam Brigagao,2,Desde o inicio tive problemas com o uso do meu...,18.6.0,2025-05-26 20:21:27,"Olá, Miriam! Acesse claro.com.br/appminhaclaro...",2025-05-27 09:53:28
4,Claro,Android,Wilmar Carvalho Dos Santos,5,Claro é excelente APP pois faz recarga até mes...,18.7.0,2025-05-26 20:08:46,None,NaT


**Dados App Store** \
Parece que a extração da App Store tem um limite de 11 páginas de comentários

In [ ]:
page = 100
dados_ios = []

apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

for app, app_id in apps.items():
    for p in range(1, page + 1):  # Coletar da página 1 até o limite máximo
        url = f'https://itunes.apple.com/rss/customerreviews/page={p}/id={app_id}/sortby=mostrecent/json'
        
        try:
            response = requests.get(url)
            data = response.json()
        except Exception as e:
            print(f"Erro ao acessar página {p} do app {app}: {e}")
            break  # Sai do loop dessa app se der erro grave

        # Se não houver mais comentários, para o loop
        if "feed" not in data or "entry" not in data["feed"]:
            print(f"Sem mais comentários na página {p} para o app {app}.")
            break

        for entry in data["feed"]["entry"][1:]:  # Ignora o primeiro, que são infos do app
            dados_ios.append({
                "app": app,
                "system": "iOS",
                "user": entry['author']['name']['label'],
                "score": entry['im:rating']['label'],
                "content": entry['content']['label']
            })

        if p % 10 == 0:
            print(f"Pausa após coletar {p} páginas do app {app}...")
            time.sleep(5)

comentarios_ios = pd.DataFrame(dados_ios, columns=["app", "system", "user", "score", "content"])

comentarios_ios.head()

Sem mais comentários na página 5 para o app Claro.
Sem mais comentários na página 6 para o app TIM.
Pausa após coletar 10 páginas do app Vivo...
Erro ao acessar página 11 do app Vivo: Expecting value: line 1 column 1 (char 0)


,app,system,user,score,content
0,Claro,iOS,TchauBye,1,Tenho essa aplicativo instalado há anos e quan...
1,Claro,iOS,campssn,1,"Poluído, muito ruim, esconde informação"
2,Claro,iOS,alemdohorizonte,1,App is not at all intuitive. One thing it does...
3,Claro,iOS,Marcelo Bento da Silva,4,"Boa tarde, sou cliente Claro e não vou mudar, ..."
4,Claro,iOS,Petterson Grande,1,Ate hoje nao ha suporte para eSIM… tudo relaci...


In [ ]:
page = 100
dados_ios = []

apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

for app, app_id in apps.items():
    for p in range(1, page + 1):  # Coletar da página 1 até o limite máximo
        url = f'https://itunes.apple.com/rss/customerreviews/page={p}/id={app_id}/sortby=mostrecent/json'
        
        try:
            response = requests.get(url)
            data = response.json()
        except Exception as e:
            print(f"Erro ao acessar página {p} do app {app}: {e}")
            break  # Sai do loop dessa app se der erro grave

        # Se não houver mais comentários, para o loop
        if "feed" not in data or "entry" not in data["feed"]:
            print(f"Sem mais comentários na página {p} para o app {app}.")
            break

        for entry in data["feed"]["entry"][1:]:  # Ignora o primeiro, que são infos do app
            dados_ios.append({
                "app": app,
                "system": "iOS",
                "user": entry['author']['name']['label'],
                "score": entry['im:rating']['label'],
                "content": entry['content']['label'],
                "app_version": entry['im:version']['label']
            })

        if p % 10 == 0:
            print(f"Pausa após coletar {p} páginas do app {app}...")
            time.sleep(5)

comentarios_ios = pd.DataFrame(dados_ios, columns=["app", "system", "user", "score", "content",
                                                   "app_version"])

comentarios_ios.head()

Sem mais comentários na página 5 para o app Claro.
Sem mais comentários na página 6 para o app TIM.
Pausa após coletar 10 páginas do app Vivo...
Erro ao acessar página 11 do app Vivo: Expecting value: line 1 column 1 (char 0)


,app,system,user,score,content,title,uptaded,im:voteSum,im:version
0,Claro,iOS,TchauBye,1,Tenho essa aplicativo instalado há anos e quan...,Aplicativo instável,NaN,0,17.53.0
1,Claro,iOS,campssn,1,"Poluído, muito ruim, esconde informação",Lixo,NaN,0,17.48.0
2,Claro,iOS,alemdohorizonte,1,App is not at all intuitive. One thing it does...,Forces You To See Ads,NaN,0,17.46.0
3,Claro,iOS,Marcelo Bento da Silva,4,"Boa tarde, sou cliente Claro e não vou mudar, ...",Mensagens,NaN,0,17.27.0
4,Claro,iOS,Petterson Grande,1,Ate hoje nao ha suporte para eSIM… tudo relaci...,Sem suporte para eSIM,NaN,0,17.26.0


Atualização de dados para acomular dados

In [ ]:
import requests
import pandas as pd
import time

# ---------------------
# 1. Parâmetros
# ---------------------
page = 100
apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

# ---------------------
# 2. Carregar base existente ou criar vazia
# ---------------------
try:
    base_existente = pd.read_csv("comentarios_ios.csv")
    print(f"Base existente carregada com {len(base_existente)} registros.")
except FileNotFoundError:
    base_existente = pd.DataFrame(columns=["app", "system", "user", "score", "content", "app_version"])
    print("Nenhuma base existente encontrada. Criando nova base.")

# Criar uma coluna de chave única na base existente
if not base_existente.empty:
    base_existente["chave"] = base_existente["app"] + base_existente["user"] + base_existente["content"] + base_existente["app_version"]

# ---------------------
# 3. Coletar novos dados
# ---------------------
dados_novos = []

for app, app_id in apps.items():
    for p in range(1, page + 1):
        url = f'https://itunes.apple.com/rss/customerreviews/page={p}/id={app_id}/sortby=mostrecent/json'
        
        try:
            response = requests.get(url)
            data = response.json()
        except Exception as e:
            print(f"Erro ao acessar página {p} do app {app}: {e}")
            break  # Sai do loop se erro

        if "feed" not in data or "entry" not in data["feed"]:
            print(f"Sem mais comentários na página {p} para o app {app}.")
            break

        for entry in data["feed"]["entry"][1:]:
            comentario = {
                "app": app,
                "system": "iOS",
                "user": entry['author']['name']['label'],
                "score": entry['im:rating']['label'],
                "content": entry['content']['label'],
                "app_version": entry['im:version']['label']
            }
            comentario["chave"] = comentario["app"] + comentario["user"] + comentario["content"] + comentario["app_version"]

            # Checar se já existe na base
            if base_existente.empty or comentario["chave"] not in base_existente["chave"].values:
                dados_novos.append(comentario)

        if p % 10 == 0:
            print(f"Pausa após coletar {p} páginas do app {app}...")
            time.sleep(5)

# ---------------------
# 4. Atualizar a base
# ---------------------
if dados_novos:
    novos_df = pd.DataFrame(dados_novos)
    print(f"{len(novos_df)} novos comentários encontrados.")

    # Concatenar e remover coluna auxiliar
    base_atualizada = pd.concat([base_existente, novos_df], ignore_index=True)
    base_atualizada = base_atualizada.drop(columns=["chave"])
    
    # Salvar
    base_atualizada.to_csv("comentarios_ios.csv", index=False)
    print("Base atualizada salva com sucesso.")
else:
    print("Nenhum novo comentário encontrado.")

# ---------------------
# 5. Visualizar
# ---------------------
if not base_existente.empty:
    print(base_existente.head())


In [9]:
comentarios_ios.shape

(898, 5)

In [8]:
comentarios = pd.concat([comentarios_android, comentarios_ios], ignore_index=True)
comentarios_android.shape, comentarios_ios.shape, comentarios.shape

((3000, 5), (21, 5), (3021, 5))